In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
sys.path.append("../src/")

In [3]:
from utilities import *

In [4]:
pd.set_option("display.max_columns", None)

In [5]:
filepath = os.path.join(dir_dict["preprocessed"], f"agg_processed.parquet")
agg = pd.read_parquet(filepath)

In [6]:
agg

,event_date,fight_id,fighter_id,fight_fighter_win,fighter_height_inches,fighter_reach_inches,opponent_id,fighter_age,fighter_height_reach_prod,fighter_cumsum_wins,fighter_cumsum_losses,fighter_cumsum_winrate,fighter_cummean_overall_kd,fighter_cummean_round1_kd,fighter_cummean_round2_kd,fighter_cummean_overall_sig_str_%,fighter_cummean_round1_sig_str_%,fighter_cummean_round2_sig_str_%,fighter_cummean_overall_td_%,fighter_cummean_round1_td_%,fighter_cummean_round2_td_%,fighter_cummean_overall_sub_att,fighter_cummean_round1_sub_att,fighter_cummean_round2_sub_att,fighter_cummean_overall_rev,fighter_cummean_round1_rev,fighter_cummean_round2_rev,fighter_cummean_overall_sig_str_%_ss,fighter_cummean_round1_sig_str_%_ss,fighter_cummean_round2_sig_str_%_ss,fighter_cummean_round3_kd,fighter_cummean_round3_sig_str_%,fighter_cummean_round3_td_%,fighter_cummean_round3_sub_att,fighter_cummean_round3_rev,fighter_cummean_round3_sig_str_%_ss,fighter_cummean_overall_sig_str_landed,fighter_cummean_overall_sig_str_total,fighter_cummean_round1_sig_str_landed,fighter_cummean_round1_sig_str_total,fighter_cummean_round2_sig_str_landed,fighter_cummean_round2_sig_str_total,fighter_cummean_overall_total_str_landed,fighter_cummean_overall_total_str_total,fighter_cummean_round1_total_str_landed,fighter_cummean_round1_total_str_total,fighter_cummean_round2_total_str_landed,fighter_cummean_round2_total_str_total,fighter_cummean_overall_td_landed,fighter_cummean_overall_td_total,fighter_cummean_round1_td_landed,fighter_cummean_round1_td_total,fighter_cummean_round2_td_landed,fighter_cummean_round2_td_total,fighter_cummean_overall_sig_str_ss_landed,fighter_cummean_overall_sig_str_ss_total,fighter_cummean_round1_sig_str_ss_landed,fighter_cummean_round1_sig_str_ss_total,fighter_cummean_round2_sig_str_ss_landed,fighter_cummean_round2_sig_str_ss_total,fighter_cummean_overall_head_ss_landed,fighter_cummean_overall_head_ss_total,fighter_cummean_round1_head_ss_landed,fighter_cummean_round1_head_ss_total,fighter_cummean_round2_head_ss_landed,fighter_cummean_round2_head_ss_total,fighter_cummean_overall_body_ss_landed,fighter_cummean_overall_body_ss_total,fighter_cummean_round1_body_ss_landed,fighter_cummean_round1_body_ss_total,fighter_cummean_round2_body_ss_landed,fighter_cummean_round2_body_ss_total,fighter_cummean_overall_leg_ss_landed,fighter_cummean_overall_leg_ss_total,fighter_cummean_round1_leg_ss_landed,fighter_cummean_round1_leg_ss_total,fighter_cummean_round2_leg_ss_landed,fighter_cummean_round2_leg_ss_total,fighter_cummean_overall_distance_ss_landed,fighter_cummean_overall_distance_ss_total,fighter_cummean_round1_distance_ss_landed,fighter_cummean_round1_distance_ss_total,fighter_cummean_round2_distance_ss_landed,fighter_cummean_round2_distance_ss_total,fighter_cummean_overall_clinch_ss_landed,fighter_cummean_overall_clinch_ss_total,fighter_cummean_round1_clinch_ss_landed,fighter_cummean_round1_clinch_ss_total,fighter_cummean_round2_clinch_ss_landed,fighter_cummean_round2_clinch_ss_total,fighter_cummean_overall_ground_ss_landed,fighter_cummean_overall_ground_ss_total,fighter_cummean_round1_ground_ss_landed,fighter_cummean_round1_ground_ss_total,fighter_cummean_round2_ground_ss_landed,fighter_cummean_round2_ground_ss_total,fighter_cummean_round3_sig_str_landed,fighter_cummean_round3_sig_str_total,fighter_cummean_round3_total_str_landed,fighter_cummean_round3_total_str_total,fighter_cummean_round3_td_landed,fighter_cummean_round3_td_total,fighter_cummean_round3_sig_str_ss_landed,fighter_cummean_round3_sig_str_ss_total,fighter_cummean_round3_head_ss_landed,fighter_cummean_round3_head_ss_total,fighter_cummean_round3_body_ss_landed,fighter_cummean_round3_body_ss_total,fighter_cummean_round3_leg_ss_landed,fighter_cummean_round3_leg_ss_total,fighter_cummean_round3_distance_ss_landed,fighter_cummean_round3_distance_ss_total,fighter_cummean_round3_clinch_ss_landed,fighter_cummean_round3_clinch_ss_total,fighter_cummean_round3_ground_ss_landed,fighter_cummean_round3_gr

In [7]:
completed_fights = agg[["event_date", "fight_id", "fighter_id", "opponent_id", "fight_fighter_win"]]

In [8]:
df = completed_fights.merge(agg.drop(["opponent_id", "fight_fighter_win"], axis=1), how="left", on="fighter_id")

In [9]:
df = df.loc[df["event_date_x"] > df["event_date_y"]].reset_index(drop=True)

In [10]:
df.shape

(68117, 288)

In [11]:
def get_latest_data(df):
    return df.loc[df["event_date_y"] == df["event_date_y"].max()]

In [12]:
df = df.groupby(["fight_id_x", "fighter_id"]).apply(get_latest_data)

In [13]:
df = df.reset_index([0, 1], drop=True).sort_index().reset_index(drop=True)

In [14]:
df = df.drop(["event_date_y", "fight_id_y"], axis=1)\
        .rename({"event_date_x": "event_date", "fight_id_x":"fight_id"},axis=1)

In [15]:
df

,event_date,fight_id,fighter_id,opponent_id,fight_fighter_win,fighter_height_inches,fighter_reach_inches,fighter_age,fighter_height_reach_prod,fighter_cumsum_wins,fighter_cumsum_losses,fighter_cumsum_winrate,fighter_cummean_overall_kd,fighter_cummean_round1_kd,fighter_cummean_round2_kd,fighter_cummean_overall_sig_str_%,fighter_cummean_round1_sig_str_%,fighter_cummean_round2_sig_str_%,fighter_cummean_overall_td_%,fighter_cummean_round1_td_%,fighter_cummean_round2_td_%,fighter_cummean_overall_sub_att,fighter_cummean_round1_sub_att,fighter_cummean_round2_sub_att,fighter_cummean_overall_rev,fighter_cummean_round1_rev,fighter_cummean_round2_rev,fighter_cummean_overall_sig_str_%_ss,fighter_cummean_round1_sig_str_%_ss,fighter_cummean_round2_sig_str_%_ss,fighter_cummean_round3_kd,fighter_cummean_round3_sig_str_%,fighter_cummean_round3_td_%,fighter_cummean_round3_sub_att,fighter_cummean_round3_rev,fighter_cummean_round3_sig_str_%_ss,fighter_cummean_overall_sig_str_landed,fighter_cummean_overall_sig_str_total,fighter_cummean_round1_sig_str_landed,fighter_cummean_round1_sig_str_total,fighter_cummean_round2_sig_str_landed,fighter_cummean_round2_sig_str_total,fighter_cummean_overall_total_str_landed,fighter_cummean_overall_total_str_total,fighter_cummean_round1_total_str_landed,fighter_cummean_round1_total_str_total,fighter_cummean_round2_total_str_landed,fighter_cummean_round2_total_str_total,fighter_cummean_overall_td_landed,fighter_cummean_overall_td_total,fighter_cummean_round1_td_landed,fighter_cummean_round1_td_total,fighter_cummean_round2_td_landed,fighter_cummean_round2_td_total,fighter_cummean_overall_sig_str_ss_landed,fighter_cummean_overall_sig_str_ss_total,fighter_cummean_round1_sig_str_ss_landed,fighter_cummean_round1_sig_str_ss_total,fighter_cummean_round2_sig_str_ss_landed,fighter_cummean_round2_sig_str_ss_total,fighter_cummean_overall_head_ss_landed,fighter_cummean_overall_head_ss_total,fighter_cummean_round1_head_ss_landed,fighter_cummean_round1_head_ss_total,fighter_cummean_round2_head_ss_landed,fighter_cummean_round2_head_ss_total,fighter_cummean_overall_body_ss_landed,fighter_cummean_overall_body_ss_total,fighter_cummean_round1_body_ss_landed,fighter_cummean_round1_body_ss_total,fighter_cummean_round2_body_ss_landed,fighter_cummean_round2_body_ss_total,fighter_cummean_overall_leg_ss_landed,fighter_cummean_overall_leg_ss_total,fighter_cummean_round1_leg_ss_landed,fighter_cummean_round1_leg_ss_total,fighter_cummean_round2_leg_ss_landed,fighter_cummean_round2_leg_ss_total,fighter_cummean_overall_distance_ss_landed,fighter_cummean_overall_distance_ss_total,fighter_cummean_round1_distance_ss_landed,fighter_cummean_round1_distance_ss_total,fighter_cummean_round2_distance_ss_landed,fighter_cummean_round2_distance_ss_total,fighter_cummean_overall_clinch_ss_landed,fighter_cummean_overall_clinch_ss_total,fighter_cummean_round1_clinch_ss_landed,fighter_cummean_round1_clinch_ss_total,fighter_cummean_round2_clinch_ss_landed,fighter_cummean_round2_clinch_ss_total,fighter_cummean_overall_ground_ss_landed,fighter_cummean_overall_ground_ss_total,fighter_cummean_round1_ground_ss_landed,fighter_cummean_round1_ground_ss_total,fighter_cummean_round2_ground_ss_landed,fighter_cummean_round2_ground_ss_total,fighter_cummean_round3_sig_str_landed,fighter_cummean_round3_sig_str_total,fighter_cummean_round3_total_str_landed,fighter_cummean_round3_total_str_total,fighter_cummean_round3_td_landed,fighter_cummean_round3_td_total,fighter_cummean_round3_sig_str_ss_landed,fighter_cummean_round3_sig_str_ss_total,fighter_cummean_round3_head_ss_landed,fighter_cummean_round3_head_ss_total,fighter_cummean_round3_body_ss_landed,fighter_cummean_round3_body_ss_total,fighter_cummean_round3_leg_ss_landed,fighter_cummean_round3_leg_ss_total,fighter_cummean_round3_distance_ss_landed,fighter_cummean_round3_distance_ss_total,fighter_cummean_round3_clinch_ss_landed,fighter_cummean_round3_clinch_ss_total,fighter_cummean_round3_ground_ss_landed,fighter_cummean_round3_gr

In [16]:
opponent_renamer = lambda c: c.replace("fighter_", "opponent_")
df = df.merge(agg.drop(["fight_fighter_win", "opponent_id"], axis=1)\
                   .rename(opponent_renamer, axis=1), how="left", on="opponent_id")

In [17]:
df = df.dropna(axis=0)

In [18]:
df = df.loc[df["event_date_x"] > df["event_date_y"]].reset_index(drop=True)

In [19]:
df.shape

(61832, 569)

In [20]:
def get_latest_data(df):
    return df.loc[df["event_date_y"] == df["event_date_y"].max()]

In [21]:
df = df.groupby(["fight_id_x", "opponent_id"]).apply(get_latest_data)

In [22]:
df = df.reset_index([0, 1], drop=True).sort_index().reset_index(drop=True)

In [23]:
df = df.drop(["event_date_y", "fight_id_y"], axis=1)\
        .rename({"event_date_x": "event_date", "fight_id_x":"fight_id"},axis=1)

In [24]:
df

,event_date,fight_id,fighter_id,opponent_id,fight_fighter_win,fighter_height_inches,fighter_reach_inches,fighter_age,fighter_height_reach_prod,fighter_cumsum_wins,fighter_cumsum_losses,fighter_cumsum_winrate,fighter_cummean_overall_kd,fighter_cummean_round1_kd,fighter_cummean_round2_kd,fighter_cummean_overall_sig_str_%,fighter_cummean_round1_sig_str_%,fighter_cummean_round2_sig_str_%,fighter_cummean_overall_td_%,fighter_cummean_round1_td_%,fighter_cummean_round2_td_%,fighter_cummean_overall_sub_att,fighter_cummean_round1_sub_att,fighter_cummean_round2_sub_att,fighter_cummean_overall_rev,fighter_cummean_round1_rev,fighter_cummean_round2_rev,fighter_cummean_overall_sig_str_%_ss,fighter_cummean_round1_sig_str_%_ss,fighter_cummean_round2_sig_str_%_ss,fighter_cummean_round3_kd,fighter_cummean_round3_sig_str_%,fighter_cummean_round3_td_%,fighter_cummean_round3_sub_att,fighter_cummean_round3_rev,fighter_cummean_round3_sig_str_%_ss,fighter_cummean_overall_sig_str_landed,fighter_cummean_overall_sig_str_total,fighter_cummean_round1_sig_str_landed,fighter_cummean_round1_sig_str_total,fighter_cummean_round2_sig_str_landed,fighter_cummean_round2_sig_str_total,fighter_cummean_overall_total_str_landed,fighter_cummean_overall_total_str_total,fighter_cummean_round1_total_str_landed,fighter_cummean_round1_total_str_total,fighter_cummean_round2_total_str_landed,fighter_cummean_round2_total_str_total,fighter_cummean_overall_td_landed,fighter_cummean_overall_td_total,fighter_cummean_round1_td_landed,fighter_cummean_round1_td_total,fighter_cummean_round2_td_landed,fighter_cummean_round2_td_total,fighter_cummean_overall_sig_str_ss_landed,fighter_cummean_overall_sig_str_ss_total,fighter_cummean_round1_sig_str_ss_landed,fighter_cummean_round1_sig_str_ss_total,fighter_cummean_round2_sig_str_ss_landed,fighter_cummean_round2_sig_str_ss_total,fighter_cummean_overall_head_ss_landed,fighter_cummean_overall_head_ss_total,fighter_cummean_round1_head_ss_landed,fighter_cummean_round1_head_ss_total,fighter_cummean_round2_head_ss_landed,fighter_cummean_round2_head_ss_total,fighter_cummean_overall_body_ss_landed,fighter_cummean_overall_body_ss_total,fighter_cummean_round1_body_ss_landed,fighter_cummean_round1_body_ss_total,fighter_cummean_round2_body_ss_landed,fighter_cummean_round2_body_ss_total,fighter_cummean_overall_leg_ss_landed,fighter_cummean_overall_leg_ss_total,fighter_cummean_round1_leg_ss_landed,fighter_cummean_round1_leg_ss_total,fighter_cummean_round2_leg_ss_landed,fighter_cummean_round2_leg_ss_total,fighter_cummean_overall_distance_ss_landed,fighter_cummean_overall_distance_ss_total,fighter_cummean_round1_distance_ss_landed,fighter_cummean_round1_distance_ss_total,fighter_cummean_round2_distance_ss_landed,fighter_cummean_round2_distance_ss_total,fighter_cummean_overall_clinch_ss_landed,fighter_cummean_overall_clinch_ss_total,fighter_cummean_round1_clinch_ss_landed,fighter_cummean_round1_clinch_ss_total,fighter_cummean_round2_clinch_ss_landed,fighter_cummean_round2_clinch_ss_total,fighter_cummean_overall_ground_ss_landed,fighter_cummean_overall_ground_ss_total,fighter_cummean_round1_ground_ss_landed,fighter_cummean_round1_ground_ss_total,fighter_cummean_round2_ground_ss_landed,fighter_cummean_round2_ground_ss_total,fighter_cummean_round3_sig_str_landed,fighter_cummean_round3_sig_str_total,fighter_cummean_round3_total_str_landed,fighter_cummean_round3_total_str_total,fighter_cummean_round3_td_landed,fighter_cummean_round3_td_total,fighter_cummean_round3_sig_str_ss_landed,fighter_cummean_round3_sig_str_ss_total,fighter_cummean_round3_head_ss_landed,fighter_cummean_round3_head_ss_total,fighter_cummean_round3_body_ss_landed,fighter_cummean_round3_body_ss_total,fighter_cummean_round3_leg_ss_landed,fighter_cummean_round3_leg_ss_total,fighter_cummean_round3_distance_ss_landed,fighter_cummean_round3_distance_ss_total,fighter_cummean_round3_clinch_ss_landed,fighter_cummean_round3_clinch_ss_total,fighter_cummean_round3_ground_ss_landed,fighter_cummean_round3_gr

In [25]:
corrs = df.corr()["fight_fighter_win"]

In [26]:
df[["fight_fighter_win", "fighter_rollsum1_wins"]]

,fight_fighter_win,fighter_rollsum1_wins
0,0,1.0
1,0,1.0
2,1,1.0
3,1,1.0
4,0,1.0
...,...,...
9141,1,0.0
9142,0,0.0
9143,0,1.0
9144,0,1.0


In [27]:
corrs.sort_values().tail(10)

fighter_cummean_overall_td_landed                     0.070171
opponent_rollsum5_losses                              0.072270
opponent_opponents_cummean_round3_ctrl_seconds        0.072578
opponent_opponents_cummean_round3_total_str_landed    0.072616
fighter_cummean_round3_td_landed                      0.073237
opponent_opponents_cummean_round2_total_str_landed    0.077457
opponent_age                                          0.100577
fight_fighter_win                                     1.000000
fighter_stance_sideways                                    NaN
opponent_stance_sideways                                   NaN
Name: fight_fighter_win, dtype: float64

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler

In [32]:
def model_performance(df):
    df_num = df.select_dtypes(["number", "bool"]).copy(deep=True)
    bool_cols = df_num.select_dtypes("bool").columns
    df_num[bool_cols] = df_num[bool_cols].astype("uint8")
    df_num = df_num.fillna(0)

    to_drop = [col for col in df_num.columns if "fight_" in col] + ["fighter_id", "opponent_id"]
    to_drop.remove("fight_fighter_win")
    df_num = df_num.drop(to_drop, axis=1)

    X, y = df_num.drop("fight_fighter_win", axis=1), df_num["fight_fighter_win"]
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    return np.mean(cross_val_score(LogisticRegression(random_state=42), X, y, cv=5, n_jobs=cpu_count()))

In [33]:
model_performance(df)

/home/alancmathew/miniconda3/envs/ufc-fight-predictor/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/alancmathew/miniconda3/envs/ufc-fight-predictor/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

0.6161162449545424